In [1]:
import numpy as np 
import matplotlib.pyplot as plt
from scipy.stats import norm, binom
from scipy.optimize import minimize

# 1. Cargamos los datos

In [2]:
data=np.array([[1,20], [6,12],[6,18],[5,20]])
N_trials=20

# 2. Early strong fusion model

In [31]:
def pyscometric(I, c, sigma):
    return norm.cdf((I-c)/sigma)

In [4]:
def weight(sigma_a, sigma_v):
    wa= sigma_v**2/(sigma_a**2 + sigma_v**2)
    return wa
def mu_av_trans(mu_a, mu_v, sigma_a, sigma_v):
    wa=weight(sigma_a, sigma_v)
    t=wa*mu_a +(1-wa)*mu_v
    return t

def sigma_av_trans(sigma_a, sigma_v):
    wa=weight(sigma_a, sigma_v)
    t=wa**2*sigma_a**2 + (1-wa)**2*sigma_v**2
    t=np.sqrt(t)
    return t
def pyscometric_audiovisual(mu_av, sigma_av):
    return norm.cdf(mu_av/sigma_av)

In [40]:
def negative_log_likelihood(params, data, N_trial):
    mu_a_tilde = np.array([0,1]) - params[0]
    mu_v_tilde = np.array([0,1]) - params[1]
    sigma_a = np.exp(params[2])
    sigma_v = np.exp(params[3])
    p_est = np.zeros((4, 2))

    p_est[0, :] = norm.cdf(mu_a_tilde / sigma_a)
    p_est[1, :] = norm.cdf(mu_v_tilde / sigma_v)
    w_a = sigma_v**2 / (sigma_a**2 + sigma_v**2)
    sigma_av = np.sqrt((sigma_a**2 * sigma_v**2) / (sigma_a**2 + sigma_v**2))

    for a in range(2):
        for v in range(2):
            mu_av_tilde = w_a * mu_a_tilde[a] + (1 - w_a) * mu_v_tilde[v]
            p_est[v + 2, a] = norm.cdf(mu_av_tilde / sigma_av)
    NegLL = 0
    for r in range(4):
        for c in range(2):
            NegLL -= np.log(binom.pmf(data[r, c], N_trial, p_est[r, c]))
    return NegLL

In [41]:
data = np.array([[1, 20], [6, 12], [6, 18], [5, 20]])
N_trial=20
Nparams = 4
params0 = np.random.rand(Nparams) - 0.5

options = {'maxiter': 1e5, 'disp': False}

result = minimize(negative_log_likelihood, params0, args=(data,N_trial), options=options)

params = result.x
c_a = params[0]
c_v = params[1]
sigma_a = np.exp(params[2])
sigma_v = np.exp(params[3])
NegLL = result.fun

print(f"c_a: {c_a}")
print(f"c_v: {c_v}")
print(f"sigma_a: {sigma_a}")
print(f"sigma_v: {sigma_v}")
print(f"NegLL: {NegLL}")

c_a: 0.30673862754524633
c_v: 0.6048262945007794
sigma_a: 0.3598265558921032
sigma_v: 1.19419726219342
NegLL: 13.397682943464504


In [7]:
NLL=negative_log_likelihood(optimized_parameters.x, data, N_trials)
print(NLL)

27.949215203768595


In [34]:
solucion_profe=[1.3067386537825676,0.3598265252980718,1.6048261189683979, 1.1941963731473209]
NLL=negative_log_likelihood(solucion_profe, data, N_trials)
print(NLL)

mu [1 2]
mu_a[1 2]
c_a 1.3067386537825676
mu_a_gorro[-0.30673865  0.69326135]
mu_v_gorro[-0.60482612  0.39517388]
[0.16794119287629744, 0.2355325220501578]
[0.9553041282458906, 0.9738223502459005]
68.51091109001565


# 2. Strong fusion probability matching model

In [8]:
def softmax(z,N_trial):
    return np.exp(z/N_trial)/(np.exp(z/N_trial)+1) #Ya que solo hay dos response probabilities

In [9]:
def prob_av(z_a, z_v, N_trial):
    num=softmax(z_a, N_trial)*softmax(z_v, N_trial)
    dem=num + softmax((N_trial-z_a),N_trial)*softmax((N_trial -z_v),N_trial)
    return num/dem

In [43]:
def negative_log_likelihood(params, data, N_trial):
    pa = np.exp(params[0:2]) / (np.exp(params[0:2]) + 1)
    pv = np.exp(params[2:4]) / (np.exp(params[2:4]) + 1)

    pav = np.zeros((2, 2))

    for a in range(2):
        for v in range(2):
            pav[v, a] = pa[a] * pv[v] / (pa[a] * pv[v] + (1 - pa[a]) * (1 - pv[v]))

    p_est = np.vstack([pa, pv, pav])

    NegLL = 0
    for r in range(4):
        for c in range(2):
            NegLL -= np.log(binom.pmf(data[r, c], N_trial, p_est[r, c]))

    return NegLL

In [44]:
data = np.array([[1, 20], [6, 12], [6, 18], [5, 20]])
Nparams = 4
params0 = np.random.rand(Nparams) - 0.5

options = {'maxiter': 1e5, 'disp': False}

result = minimize(negative_log_likelihood, params0, args=(data,N_trial), options=options)

params = result.x
pa = np.exp(params[0:2]) / (np.exp(params[0:2]) + 1)
pv = np.exp(params[2:4]) / (np.exp(params[2:4]) + 1)
NegLL = result.fun

print(f"pa: {pa}")
print(f"pv: {pv}")
print(f"NegLL: {NegLL}")
print(f"Params{params}")

pa: [0.1926458  0.96777303]
pv: [0.40613832 0.60427944]
NegLL: 14.35414112007535
Params[-1.4329092   3.40219406 -0.37995263  0.42332844]


In [46]:
NLL=negative_log_likelihood(result.x, data, N_trials)
print(NLL)

14.35414112007535
